In [1]:
import json
import os
import csv
import shutil
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [5]:
root_directory = "/Users/JB/Rotation_bkslab/nonhits_ampc/json_files"  
output_directory = "/Users/JB/Rotation_bkslab/nonhits_ampc/JSON"

os.makedirs(output_directory, exist_ok=True)

for subdir, _, files in os.walk(root_directory):
    for file in files:
        if file.endswith("_confidences.json"):
            source_path = os.path.join(subdir, file)
            destination_path = os.path.join(output_directory, file)

            new_filename = file.replace("_confidences", "")
            destination_path = os.path.join(output_directory, new_filename)

            # Move the file to the new directory
            shutil.move(source_path, destination_path)

print("\n All matching files have been moved to:", output_directory)


 All matching files have been moved to: /Users/JB/Rotation_bkslab/nonhits_ampc/JSON


In [6]:
# Input and Output directories
input_dir = "/Users/JB/Rotation_bkslab/nonhits_ampc/JSON"
output_dir = "/Users/JB/Rotation_bkslab/nonhits_ampc/JSON_ligands"
csv_file = "/Users/JB/Rotation_bkslab/nonhits_ampc/ligand_plddt_nonhits.csv"
os.makedirs(output_dir, exist_ok=True)

total_plddt = []

for json_file in glob.glob(os.path.join(input_dir, "*.json")):
    with open(json_file, "r") as file:
        data = json.load(file)
    atom_chain_ids = data.get("atom_chain_ids", [])
    atom_plddts = data.get("atom_plddts", [])
    ref_prefix = os.path.splitext((os.path.basename(json_file)))[0]

    filtered_chain_ids = []
    filtered_plddts = []
    
    for chain_id, plddt in zip(atom_chain_ids, atom_plddts):
        if chain_id == "B":
            filtered_chain_ids.append(chain_id)
            filtered_plddts.append(plddt)
    
    average_plddt = sum(filtered_plddts) / len(filtered_plddts) if filtered_plddts else 0
    
    data["atom_chain_ids"] = filtered_chain_ids
    data["atom_plddts"] = filtered_plddts
    data["average_plddt"] = average_plddt
    total_plddt.append([ref_prefix, average_plddt])
    
    # Generate the output file path
    output_file_path = os.path.join(output_dir, os.path.basename(json_file))

    # Save the modified JSON
    with open(output_file_path, "w") as output_file:
        json.dump(data, output_file, indent=4)

# Save results to CSV
with open(csv_file, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Ligand", "L-pLDDT"])
    writer.writerows(total_plddt)

print(f"[INFO] L-pLDDT results saved to {csv_file}")

[INFO] L-pLDDT results saved to /Users/JB/Rotation_bkslab/nonhits_ampc/ligand_plddt_nonhits.csv
